https://stackoverflow.com/questions/53252181/python-worker-failed-to-connect-back

Schema: https://stackoverflow.com/questions/45276427/unable-connect-to-docker-container-outside-docker-host

Config: https://stackoverflow.com/questions/72113678/hdfs-config-for-pyspark

IP docker: https://stackoverflow.com/questions/61432807/hadoop-spark-there-are-1-datanodes-running-and-1-nodes-are-excluded-in-th

Hosts windows: https://fixergeek.com/fixes/how-to-assign-hostname-to-ip-address-in-windows#:~:text=Press%20the%20Windows%20button%20%2B,key.%20IP%20configuration%20of%20Windows

Windows 11: https://www.sysgeeker.com/blog/how-to-add-to-host-file-windows-11.html

C:\Windows\System32\drivers\etc\hosts

hdfs dfsadmin -report

https://ruslanmv.com/blog/How-to-install-Hadoop-on-Windows

In [1]:
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch
from pyspark.sql.types import *
import findspark
findspark.init()

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('error', category=DeprecationWarning)

In [3]:
client = Elasticsearch("http://localhost:9200")

client.ping()

True

In [4]:
sparkSession = SparkSession.builder.appName("csv4").getOrCreate()

24/03/11 17:41:14 WARN Utils: Your hostname, gwynbleidd-NBR-WAX9 resolves to a loopback address: 127.0.1.1; using 192.168.96.4 instead (on interface wlp0s20f3)
24/03/11 17:41:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 17:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sparkSession

In [6]:
searchBody = {
  "size": 9999,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="master", body=searchBody)
masters = result['hits']['hits']
result = client.search(index="order", body=searchBody)
orders = result['hits']['hits']

In [7]:
masters[:1]

[{'_index': 'master',
  '_type': '_doc',
  '_id': '563268',
  '_score': 1.0,
  '_source': {'master_id': 563268,
   'master_desc': 'Милица Руслановна Фомина, Тип Образования: Среднее.',
   'master_feedbacks': ['ворчливый, медлительный.',
    'ворчливый, медлительный.',
    'медлительный, ворчливый.']}}]

In [8]:
MasterSchema = StructType([
    StructField("master_id", IntegerType(), False),
    StructField("master_desc", StringType(), False),
    StructField("master_feedbacks", StringType(), False)
])
CustomerSchema = StructType([
    StructField("order_customer_id", IntegerType(), False),
    StructField("order_customer_desc", StringType(), False)
])
OrderSchema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("order_customer_id", IntegerType(), False),
    # StructField("order_customer_desc", StringType(), False),
    StructField("order_details_desc", StringType(), False),
    StructField("order_due_date", StringType(), False),
    StructField("order_fact_completion_date", StringType(), False),
    StructField("order_parts", StringType(), False),
    StructField("repair_types", StringType(), False),
    StructField("order_price", FloatType(), False),
    StructField("order_master_id", IntegerType(), False)
])

In [9]:
MasterTable = []
CustomerTable = []
OrderTable = []

In [10]:
CustomerSet = set()

24/03/11 17:41:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [11]:
for master in masters:
    MasterTable.append((
        master['_source']['master_id'],
        master['_source']['master_desc'],
        str(master['_source']['master_feedbacks'])
    ))
for order in orders:
    OrderTable.append((
        order['_source']['order_id'],
        order['_source']['order_date'],
        order['_source']['order_customer_id'],
        order['_source']['order_details_desc'],
        order['_source']['order_due_date'],
        order['_source']['order_fact_completion_date'],
        str(order['_source']['order_parts']),
        str(order['_source']['repair_types']),
        order['_source']['order_price'],
        order['_source']['order_master_id']
    ))
    if not order['_source']['order_id'] in CustomerSet:
        CustomerSet.add(order['_source']['order_id'])
        CustomerTable.append((
            order['_source']['order_customer_id'],
            order['_source']['order_customer_desc']
        ))

In [12]:
MasterDF = sparkSession.createDataFrame(MasterTable, MasterSchema)
OrderDF = sparkSession.createDataFrame(OrderTable, OrderSchema)
CustomerDF = sparkSession.createDataFrame(CustomerTable,CustomerSchema)

In [13]:
MasterDF.show()

+---------+--------------------+--------------------+
|master_id|         master_desc|    master_feedbacks|
+---------+--------------------+--------------------+
|   563268|Милица Руслановна...|['ворчливый, медл...|
|   871908|Козлов Юрий Фомич...|['ворчливый, акку...|
|   164475|Юлий Дмитриевич К...|['аккуратный, вор...|
|   464209|Ия Тарасовна Степ...|['аккуратный, мед...|
|    22653|Акулина Рудольфов...|['ворчливый, акку...|
|   282806|Маслов Емельян Дм...|['ворчливый, медл...|
|   736774|Владимирова Анжел...|['аккуратный, вор...|
|   300714|Вероника Петровна...|['ворчливый, медл...|
|   405064|Хохлов Олег Харла...|['ворчливый, медл...|
|   257458|Шестакова Наина В...|['аккуратный, вор...|
|   814777|Шарова Жанна Арка...|['ворчливый, медл...|
|   220028|Зиновий Димитриев...|['медлительный, в...|
|   397809|Тарасова Фёкла Ва...|['медлительный, в...|
|   847121|Жуков Ладимир Все...|['медлительный, в...|
|   229678|Евпраксия Альберт...|['аккуратный, мед...|
|   592194|Котова Валентина 

In [14]:
# OrderDF.show()

In [15]:
# CustomerDF.show()

In [14]:
MasterDF.write.csv(path='hdfs://0.0.0.0:9000/master.csv',mode='overwrite', header=True)
OrderDF.write.csv(path='hdfs://0.0.0.0:9000/order.csv',mode='overwrite', header=True)
CustomerDF.write.csv(path='hdfs://0.0.0.0:9000/customer.csv',mode='overwrite', header=True)

In [15]:
data = sparkSession.read.load("hdfs://localhost:9000/master.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("master")
data = sparkSession.read.load("hdfs://localhost:9000/order.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("order")
data = sparkSession.read.load("hdfs://localhost:9000/customer.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("customer")

In [16]:
result = sparkSession.sql("""select m.master_id, m.master_desc, o.order_id, o.order_date, o.order_due_date, o.order_fact_completion_date, c.order_customer_id, c.order_customer_desc 
from master m JOIN order o ON o.order_master_id = m.master_id LEFT JOIN customer c ON o.order_customer_id = c.order_customer_id
where o.order_fact_completion_date > o.order_due_date
""")
result.show(20, False)

+---------+----------------------------------------------------------+--------+----------+--------------+--------------------------+-----------------+---------------------------------------------------------------+
|master_id|master_desc                                               |order_id|order_date|order_due_date|order_fact_completion_date|order_customer_id|order_customer_desc                                            |
+---------+----------------------------------------------------------+--------+----------+--------------+--------------------------+-----------------+---------------------------------------------------------------+
|257458   |Шестакова Наина Владиславовна, Стаж Работы: 9 Л./Г..      |131919  |2024-01-06|2024-04-10    |2024-04-27                |227431           |Имя: Маргарита Валентиновна Чернова; премиальный, постоянный.  |
|814777   |Шарова Жанна Аркадьевна, Стаж Работы: 1 Л./Г..            |141550  |2024-01-10|2024-04-03    |2024-04-27                |683623  

In [4]:
sparkSession.stop()